In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
import os

# 'problem' klasörünün içeriğini listeleyelim
os.listdir('/content/drive/MyDrive/hata/HATALI_REASON/HATALI FAZLA OLAN DATALARIN MODELLERİ/ÜRÜN GEÇ/WORD COUNT')


['hatalı_filtre_stopwords.ipynb adlı dosyanın kopyası',
 'URUNGECHATA.xlsx',
 'urun_gec_hata.xlsx',
 'urun_gec_doğru.xlsx']

In [ ]:
import pandas as pd

In [ ]:
klasor_adi= ['URUNKALİTE',
 'KUTU',
 'KAMPANYA',
 'KASA',
 'KARGO',
 'MAĞAZA','ÖDEME SORUNU','MAGAZA DÜZENİ','ÜRÜN ÇEŞİDİ AZ']

reason_adi=klasor_adi[-1]
reason_adi

'ÜRÜN ÇEŞİDİ AZ'

In [ ]:
file_path=f'/content/drive/MyDrive/hata/HATALI_REASON/HATALI FAZLA OLAN DATALARIN MODELLERİ/ÜRÜN GEÇ/WORD COUNT/URUNGECHATA.xlsx'
df = pd.read_excel(file_path)


In [ ]:
file_path=f'/content/drive/MyDrive/hata/HATALI_REASON/HATALI FAZLA OLAN DATALARIN MODELLERİ/ÜRÜN GEÇ/WORD COUNT/urun_gec_doğru.xlsx'
df_dogru = pd.read_excel(file_path)


In [ ]:
file_path=f'/content/drive/MyDrive/hata/HATALI_REASON/HATALI FAZLA OLAN DATALARIN MODELLERİ/ÜRÜN GEÇ/WORD COUNT/urun_gec_hata.xlsx'
df_hata = pd.read_excel(file_path)


In [ ]:
!pip install TurkishStemmer

In [ ]:

import nltk
import pandas as pd
import re
from nltk.corpus import stopwords
from TurkishStemmer import TurkishStemmer

# Gerekli verileri indir
nltk.download('stopwords')

# Stopwords listesi
turkish_stopwords = set(stopwords.words('turkish'))
#ozel_kelimeler = ['bir', '2', 'i', 'tl', 'yok', '1', 'dedi', 'ben']
#turkish_stopwords.update(ozel_kelimeler)

# Türkçe kökleyici
stemmer = TurkishStemmer()

# Temizleme + kökleme fonksiyonu
def temizle_ve_kokle(yorum):
    if pd.isna(yorum):
        return ""
    kelimeler = re.findall(r'\b\w+\b', yorum.lower())
    temiz_kelimeler = [
        stemmer.stem(kelime) for kelime in kelimeler if kelime not in turkish_stopwords
    ]
    return " ".join(temiz_kelimeler)

# Yeni sütun oluştur
df_dogru["Temiz_Kok_Yorum"] = df_dogru["Yorum"].apply(temizle_ve_kokle)

# İlk 5 sonucu göster
df_dogru[["Yorum", "Temiz_Kok_Yorum"]].head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Yorum,Temiz_Kok_Yorum
0,Çanta almış yanlış ürün gönderilmiş.,çanta al yanlış ürün gönderil
1,Kargom teslim edilmesi gereken tarihte henüz k...,kargom tesl edilme gere tarih henüz kargo bil ...
2,Terlik almış görseldekinden farklı ürün gelmiş...,terlik al görseldek fark ürün gel kargo geç ge...
3,2 haziranda 2 ürün sipariş ettim kargolanması ...,2 haziran 2 ürün sipariş ett kargolanma uzun s...
4,Ayakkabı aldık. Kargom yırtık geldi. Kutu ve ...,ayakkabı al kargom yır gelt kut poşet yır gelt...


In [ ]:
df["Temiz_Kok_Yorum"] = df["Yorum"].apply(temizle_ve_kokle)

# İlk 5 sonucu göster
df[["Yorum", "Temiz_Kok_Yorum"]].head()


,Yorum,Temiz_Kok_Yorum
0,Ziraat bankasının 2500 tl üzerine 250 tl puan ...,ziraat banka 2500 tl üzer 250 tl puan kampanya...
1,Yaptığım hediye çekiyle maalesef istediğim ürü...,yaptık hedi çeki maalesef istedik ürün olmadık...
2,Yapmis oldugunuz kampanyalari düzgün sekilde a...,yapmis oldug kampanyalari düzg sekil anla yazi...
3,Yapı Kredi kartımın puanını yüzde onunu alıyor...,yap kret kart puan yüz onun alıyor oys ben tam...
4,Yaklaşık 3 saat zaman ayırdı toplam 10 ad ayak...,yaklaşık 3 saat zaman ayır topla 10 ad ayakkab...


In [ ]:
df.shape

(17236, 18)

In [ ]:
#df = df[~df["Temiz_Kok_Yorum"].str.contains("çeşit|numara|çeşi|aradık", na=False)]


In [ ]:
df.shape

(17236, 18)

In [ ]:
from collections import Counter
import re

# Yorum sütunundaki tüm metinleri birleştir
tum_yorumlar = " ".join(df_dogru["Temiz_Kok_Yorum"].dropna().astype(str))

# Metni kelimelere ayır ve temizle (sadece harf içerenleri al, küçük harfe çevir)
kelimeler = re.findall(r'\b\w+\b', tum_yorumlar.lower())

# Kelime sıklıklarını say
kelime_sayaci = Counter(kelimeler)

# Sonuçları DataFrame'e çevirip en çok geçenleri görebilirsin
df_kelime_sayilari = pd.DataFrame(kelime_sayaci.items(), columns=["Kelime", "Geçme Sayısı"])
df_kelime_sayilari = df_kelime_sayilari.sort_values(by="Geçme Sayısı", ascending=False).reset_index(drop=True)

# İlk 20 kelimeyi göster (isteğe bağlı)
print(df_kelime_sayilari.head(40))


      Kelime  Geçme Sayısı
0       gelt            38
1      kargo            34
2       ürün            31
3        gün            28
4         al            23
5        geç            21
6    sipariş            19
7      bayra            18
8          2            17
9   ayakkabı            17
10      sonr            16
11      tesl            14
12  gönderil            11
13     veril            11
14    kargom            11
15      edil            10
16         i            10
17       ett             8
18       bir             8
19      sürt             8
20     süreç             7
21    numara             7
22      iade             7
23      uzun             7
24       var             7
25       haf             6
26      süre             6
27       ade             6
28         1             6
29      vert             6
30    yanlış             6
31    mağdur             5
32    müşter             5
33       önç             5
34      ulaş             5
35      aras             5
3

In [ ]:
print(df_kelime_sayilari.head(5))

    Kelime  Geçme Sayısı
0      geç            21
1  sipariş            19
2    bayra            18
3     sonr            16
4     tesl            14


In [ ]:
df_kelime_sayilari = df_kelime_sayilari.drop(index=2).reset_index(drop=True)


In [ ]:
silmek_istenen_kelimeler = ["kargo", "ürün", "gün","al"]

df_kelime_sayilari = df_kelime_sayilari[~df_kelime_sayilari["Kelime"].isin(silmek_istenen_kelimeler)]

In [ ]:
print(df_kelime_sayilari.head(40))

      Kelime  Geçme Sayısı
0        geç            21
1    sipariş            19
2       tesl            14
3   gönderil            11
4      veril            11
5     kargom            11
6       edil            10
7          i            10
8        ett             8
9        bir             8
10      sürt             8
11     süreç             7
12    numara             7
13      iade             7
14      uzun             7
15       var             7
16       haf             6
17      süre             6
18       ade             6
19         1             6
20      vert             6
21    yanlış             6
22    mağdur             5
23    müşter             5
24       önç             5
25      ulaş             5
26      aras             5
27     gelen             5
28       olt             5
29      fark             5
30         z             4
31       hal             4
32       kri             4
33         3             4
34         5             4
35        10             4
3

In [ ]:
df.shape

(17236, 18)

In [ ]:
# İlk 10 kelimeyi al
ilk_10_kelimeler = df_kelime_sayilari["Kelime"].head(5).tolist()

df = df[df["MAP"] == "Hatalı Reason Seçimi"].copy()


# Yorum sütununu küçük harfe çevirerek işle
def yorumda_geciyor_mu(yorum):
    if pd.isna(yorum):
        return False
    yorum = yorum.lower()
    return any(kelime in yorum for kelime in ilk_10_kelimeler)

# Belirtilen kelimelerden herhangi birini içeren satırları filtrele (GEÇENLERİ çıkar)
df_filtreli = df[~df["Yorum"].apply(yorumda_geciyor_mu)].reset_index(drop=True)

# Sonucu kontrol et
print(df_filtreli.head())


                                               Yorum                   MAP  \
0  Yaptığım hediye çekiyle maalesef istediğim ürü...  Hatalı Reason Seçimi   
1  Yapmis oldugunuz kampanyalari düzgün sekilde a...  Hatalı Reason Seçimi   
2  Yapı Kredi kartımın puanını yüzde onunu alıyor...  Hatalı Reason Seçimi   
3  World pay ödeme alamadılar.koç ailem world pay...  Hatalı Reason Seçimi   
4  ürünün üzerinde indirim yazmasına rağmen indir...  Hatalı Reason Seçimi   

  Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7  \
0   KAMPANYA      15429       1807     -13622   KAMPANYA       KASA   
1   KAMPANYA        NaN        NaN        NaN        743        287   
2   KAMPANYA        NaN        NaN        NaN        NaN        NaN   
3   KAMPANYA        NaN        NaN        NaN        NaN        NaN   
4   KAMPANYA        NaN        NaN        NaN        NaN        NaN   

  Unnamed: 8 Unnamed: 9 Unnamed: 10    Unnamed: 11   Unnamed: 12  Unnamed: 13  \
0     MAĞAZA      KARGO

In [ ]:
df.shape

(15429, 17)

In [ ]:
df_filtreli.shape

(12640, 17)

In [ ]:
print(df_filtreli.columns)

Index(['Yorum', 'MAP', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16'],
      dtype='object')


In [ ]:
# Excel dosyasına kaydet
df_filtreli.to_excel("10word_filtered.xlsx", index=False)
print("Hatalı veriler 10word_filtered.xlsx olarak kaydedildi.")

# Dosyayı Colab üzerinden indir
from google.colab import files
files.download("10word_filtered.xlsx")


Hatalı veriler 10word_filtered.xlsx olarak kaydedildi.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>